In [11]:
import requests
import bs4
import urllib.request as REQ

url="https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&date=20210520"
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser') 

arr=[]
for tmp in soup.select("#old_content > table > tbody > tr > td.title > div > a",limit=5):
    print(tmp.text)
    print(tmp.attrs["href"])
    url="https://movie.naver.com"+tmp.attrs["href"]
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')

    data={"title":tmp.text}
    director=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4) > p > a").text
    actors=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p").text
    grade=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(8) > p > a").text
    genre=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(1) > a").text
    nation=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(2) > a").text
    runningtime=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(3)").text
    opendate=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4)").text
    point=soup.select_one("#actualPointPersentBasic > div > span > span").text
    audience=soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(10) > div > p.count").text
    poster=soup.select_one("#content > div.article > div.mv_info_area > div.poster > a > img").attrs.get("src")
    
    print("감독",director)
    print("배우:",actors)
    print("등급:",grade)  
    print("장르:",genre)
    print("국가:",nation)
    print("시간:",runningtime)
    print("개봉:",opendate[2:6]+"."+opendate[8:13])
    print(f"평점: {point[7:11]}")
    print(f"누적관객수: {audience}")
    print(poster)
    
    data["director"]=director
    data["actors"]=actors
    data["grade"]=grade
    data["genre"]=genre
    data["nation"]=nation
    data["runningtime"]=runningtime
    data["opendate"]=opendate[2:6]+"."+opendate[8:13]
    data["point"]=point[7:11]
    
    REQ.urlretrieve(poster, f'image/movie/movie_{tmp.text.replace(":","_")}.jpg')
    
    print(data)
    arr.append(data)
    print("="*40)

분노의 질주: 더 얼티메이트
/movie/bi/mi/basic.nhn?code=189150
감독 저스틴 린
배우: 빈 디젤(도미닉 토레토), 존 시나(제이콥 토레토), 성 강(한)
등급: 12세 관람가
장르: 액션
국가: 미국
시간: 142분 
개봉: 2021.05.19
평점: 9.05
누적관객수: 1,203,909명(05.24 기준)
https://movie-phinf.pstatic.net/20210421_37/1618971733493B4ykS_JPEG/movie_image.jpg?type=m203_290_2
{'title': '분노의 질주: 더 얼티메이트', 'director': '저스틴 린', 'actors': '빈 디젤(도미닉 토레토), 존 시나(제이콥 토레토), 성 강(한)', 'grade': '12세 관람가', 'genre': '액션', 'nation': '미국', 'runningtime': '142분 ', 'opendate': '2021.05.19', 'point': '9.05'}
스파이럴
/movie/bi/mi/basic.nhn?code=191570
감독 대런 린 보우즈만
배우: 크리스 록, 사무엘 L. 잭슨, 마리솔 니콜스
등급: 청소년 관람불가
장르: 스릴러
국가: 미국
시간: 93분 
개봉: 2021.05.12
평점: 7.74
누적관객수: 124,297명(05.24 기준)
https://movie-phinf.pstatic.net/20210427_222/1619511153612Vrb5k_JPEG/movie_image.jpg?type=m203_290_2
{'title': '스파이럴', 'director': '대런 린 보우즈만', 'actors': '크리스 록, 사무엘 L. 잭슨, 마리솔 니콜스', 'grade': '청소년 관람불가', 'genre': '스릴러', 'nation': '미국', 'runningtime': '93분 ', 'opendate': '2021.05.12', 'point': '7.74'}
더 스파이
/movie/bi/mi/ba

In [8]:
# 수집된 자료를 몽고DB에 저장
# 컬렉션은 exam_movie
# (영화제목, 관람등급, 관객평점, 장르, 상영시간, 개봉일, 감독, 출연배우)

import pymongo

try:
    username = "id125"
    password = "pw125"
    host = "1.234.5.158"
    port = 37017
    dbname = "id125"
    
    conn = pymongo.MongoClient(f'mongodb://{username}:{password}@{host}:{port}/{dbname}')
    db = conn.get_database(dbname)
    collection = db.get_collection("exam_movie")  #table과 같은 개념
    collection.insert_many(arr)
except Exception as e:
    print(e)
finally:
    conn.close()

In [12]:
#hadoop에 저장
import hdfs
import json

client_hdfs = hdfs.InsecureClient('http://1.234.5.158:9870', user='hdfs')


print(arr)

client_hdfs.delete('/id125/ex_20210525.json')

with client_hdfs.write('/id125/ex_20210525.json', encoding="utf-8") as writer:
    json.dump(arr,writer)
    
# 5. 확인 크롬에서  http://1.234.5.158:9870/explorer.html#/

[{'title': '분노의 질주: 더 얼티메이트', 'director': '저스틴 린', 'actors': '빈 디젤(도미닉 토레토), 존 시나(제이콥 토레토), 성 강(한)', 'grade': '12세 관람가', 'genre': '액션', 'nation': '미국', 'runningtime': '142분 ', 'opendate': '2021.05.19', 'point': '9.05'}, {'title': '스파이럴', 'director': '대런 린 보우즈만', 'actors': '크리스 록, 사무엘 L. 잭슨, 마리솔 니콜스', 'grade': '청소년 관람불가', 'genre': '스릴러', 'nation': '미국', 'runningtime': '93분 ', 'opendate': '2021.05.12', 'point': '7.74'}, {'title': '더 스파이', 'director': '도미닉 쿡', 'actors': '베네딕트 컴버배치(그레빌 윈)', 'grade': '15세 관람가', 'genre': '스릴러', 'nation': '미국', 'runningtime': '112분 ', 'opendate': '2021.04.28', 'point': '8.78'}, {'title': '크루즈 패밀리: 뉴 에이지', 'director': '조엘 크로포드', 'actors': '엠마 스톤(이프 목소리), 라이언 레이놀즈(가이 목소리), 켈리 마리 트란(던 목소리)', 'grade': '전체 관람가', 'genre': '애니메이션', 'nation': '미국', 'runningtime': '95분 ', 'opendate': '2021.05.05', 'point': '9.35'}, {'title': '극장판 귀멸의 칼날: 무한열차편', 'director': '소토자키 하루오', 'actors': '하나에 나츠키(카마도 탄지로 목소리), 키토 아카리(카마도 네즈코 목소리)', 'grade': '15세 관람가', 'genre': '애니메이션', 'nation